In [ ]:
# this file downloads xml files from pubmed ftp
# fisrt, get a list of target files on the pubmed server
# then, for each file, do the following:
#  - access ftp and download the xml file
#  - do the parsing and output the dataframe
#  - delete the xml file (to save space on local disk)
# note: we access ftp once per file since the processing of one file can take some time, and 
# ftp might lose connection during this period of inactivation

In [1]:
import ftplib
import os
# uncomment this line on rcc !pip install --user git+http://github.com/titipata/pubmed_parser.git
import pubmed_parser as pp
import pandas as pd
import pickle

import nltk
from nltk.corpus import stopwords
import numpy as np

In [2]:
ftp = ftplib.FTP("ftp.ncbi.nlm.nih.gov")
ftp.login("anonymous", "yihanzhang@uchicago.edu")

'230 Anonymous access granted, restrictions apply'

In [3]:
ftp.cwd("/pubmed/updatefiles")
files = ftp.nlst() # get list of filenames
ftp.quit()

'221 Goodbye.'

In [4]:
files = [f for f in files if f.endswith("xml.gz")]
files # len 81

['pubmed18n0929.xml.gz',
 'pubmed18n0930.xml.gz',
 'pubmed18n0931.xml.gz',
 'pubmed18n0932.xml.gz',
 'pubmed18n0933.xml.gz',
 'pubmed18n0934.xml.gz',
 'pubmed18n0935.xml.gz',
 'pubmed18n0936.xml.gz',
 'pubmed18n0937.xml.gz',
 'pubmed18n0938.xml.gz',
 'pubmed18n0939.xml.gz',
 'pubmed18n0940.xml.gz',
 'pubmed18n0941.xml.gz',
 'pubmed18n0942.xml.gz',
 'pubmed18n0943.xml.gz',
 'pubmed18n0944.xml.gz',
 'pubmed18n0945.xml.gz',
 'pubmed18n0946.xml.gz',
 'pubmed18n0947.xml.gz',
 'pubmed18n0948.xml.gz',
 'pubmed18n0949.xml.gz',
 'pubmed18n0950.xml.gz',
 'pubmed18n0951.xml.gz',
 'pubmed18n0952.xml.gz',
 'pubmed18n0953.xml.gz',
 'pubmed18n0954.xml.gz',
 'pubmed18n0955.xml.gz',
 'pubmed18n0956.xml.gz',
 'pubmed18n0957.xml.gz',
 'pubmed18n0958.xml.gz',
 'pubmed18n0959.xml.gz',
 'pubmed18n0960.xml.gz',
 'pubmed18n0961.xml.gz',
 'pubmed18n0962.xml.gz',
 'pubmed18n0963.xml.gz',
 'pubmed18n0964.xml.gz',
 'pubmed18n0965.xml.gz',
 'pubmed18n0966.xml.gz',
 'pubmed18n0967.xml.gz',
 'pubmed18n0968.xml.gz',


In [5]:
files = ["pubmed18n1021.xml.gz", "pubmed18n1022.xml.gz", "pubmed18n1023.xml.gz", "pubmed18n1024.xml.gz"] #### debug ####
#files = ["pubmed18n0949.xml.gz"]

In [6]:
files

['pubmed18n1021.xml.gz',
 'pubmed18n1022.xml.gz',
 'pubmed18n1023.xml.gz',
 'pubmed18n1024.xml.gz']

In [7]:
# exclude stopwords (need to be editted if wanted stemmer)
def normlizeTokens(tokenLst, stopwordLst = None, stemmer = None):

    workingIter = (w.lower() for w in tokenLst if w.isalpha())

    if stemmer is not None:
        workingIter = (stemmer.stem(w) for w in workingIter)

    if stopwordLst is not None:
        workingIter = (w for w in workingIter if w not in stopwordLst)
     # return a list with the stopwords removed
    return list(workingIter)

def psycfilter(normalLst):
    result = (w for w in normalLst if w in new_dict)
    return list(result)

In [8]:
for filename in files:
    # access ftp
    ftp = ftplib.FTP("ftp.ncbi.nlm.nih.gov")
    ftp.login("anonymous", "yihanzhang@uchicago.edu")
    ftp.cwd("/pubmed/updatefiles")
    # download xml file
    with open(filename, "wb") as f:
        ftp.retrbinary("RETR " + filename, f.write)
    ftp.quit()
    # do parsing
    pubmed_parsed = pp.parse_medline_xml("./" + filename)
    pubmed_df = pd.DataFrame(pubmed_parsed)
    
    # filter out relevant rows
    new_dict = ['psycholog', 'behavior','neurosci']

    stop_words_nltk = stopwords.words('english')
    porter = nltk.stem.porter.PorterStemmer()
    snowball = nltk.stem.snowball.SnowballStemmer('english')

    # tokenize abstract
    not_delete = ~pubmed_df['delete'] 
    pubmed_df.loc[not_delete,'tokenized_abstract'] = pubmed_df.loc[not_delete,'abstract'].apply(nltk.word_tokenize) 

    # fliter out deleted=TRUE pubmed
    pubmedDF = pubmed_df[~pubmed_df['abstract'].isnull()].copy()

    # tokenize title
    pubmedDF['tokenized_title'] = pubmedDF['title'].apply(lambda x: nltk.word_tokenize(x))



    pubmedDF['normalized_abstract'] = pubmedDF['tokenized_abstract'].apply(lambda x: normlizeTokens(x, stopwordLst = stop_words_nltk, stemmer = snowball))
    pubmedDF['normalized_title'] = pubmedDF['tokenized_title'].apply(lambda x: normlizeTokens(x, stopwordLst = stop_words_nltk, stemmer = snowball))

    pubmed_new = pubmedDF[pubmedDF.abstract != ""] # exclude those without abstract

    pubmed_new['psyc_check'] = pubmed_new['normalized_abstract'].apply(lambda x: psycfilter(x))
    pubmed_new['psyc_filter'] = pubmed_new['psyc_check'].apply(lambda x: True if len(x) > 0 else False)
    pubmed_filtered = pubmed_new[pubmed_new['psyc_filter'] == False]
    
    # save df to local
    pubmed_filtered.to_csv("./intermediate_data_nonpsyc/" + filename[:-7] + ".csv")
    with open("./intermediate_data_nonpsyc/" + filename[:-7] + ".pkl", "wb") as f:
        pickle.dump(pubmed_filtered, f) # to reload: df = pd.read_pickle(filename)
    # delete xml file
    os.remove(filename)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
pubmed_df = pd.read_pickle("./intermediate_data_nonpsyc/pubmed18n1021.pkl")
pubmed_df

,abstract,affiliation,author,country,delete,issn_linking,journal,keywords,medline_ta,mesh_terms,...,pmc,pmid,pubdate,title,tokenized_abstract,tokenized_title,normalized_abstract,normalized_title,psyc_check,psyc_filter
0,Antigens shared between Streptococcus pyogenes...,"Department of Microbiology and Immunology, Uni...",LA Barnett; MW Cunningham,United States,False,0019-9567,Infection and immunity,,Infect Immun,D000199:Actins; D000251:Adenosine Triphosphata...,...,PMC257414,1386840,1992,Evidence for actinlike proteins in an M protei...,"[Antigens, shared, between, Streptococcus, pyo...","[Evidence, for, actinlike, proteins, in, an, M...","[antigen, share, streptococcus, pyogen, heart,...","[evid, actinlik, protein, strain, streptococcu...",[],False
1,Association of group A streptococci with acute...,"Department of Pathology and Microbiology, Univ...",JM Gulizia; MW Cunningham; BM McManus,United States,False,0002-9440,The American journal of pathology,,Am J Pathol,D000328:Adult; D000368:Aged; D000907:Antibodie...,...,PMC1886198,1704188,1991,Immunoreactivity of anti-streptococcal monoclo...,"[Association, of, group, A, streptococci, with...","[Immunoreactivity, of, anti-streptococcal, mon...","[associ, group, streptococci, acut, rheumat, f...","[immunoreact, monoclon, antibodi, human, heart...",[],False
3,BACKGROUND\nThe excess risk of atherosclerosis...,"Department of Epidemiology, School of Public H...",MA Austin; L Mykkänen; J Kuusisto; KL Edwards;...,United States,False,0009-7322,Circulation,,Circulation,D000368:Aged; D016022:Case-Control Studies; D0...,...,,7671360,1995,Prospective study of small LDLs as a risk fact...,"[BACKGROUND, The, excess, risk, of, atheroscle...","[Prospective, study, of, small, LDLs, as, a, r...","[background, excess, risk, atherosclerosi, amo...","[prospect, studi, small, ldls, risk, factor, d...",[],False
4,The development of autoimmunity in certain ins...,"Department of Microbiology and Immunology, Uni...",MW Cunningham; SM Antone; JM Gulizia; BM McMan...,United States,False,0027-8424,Proceedings of the National Academy of Science...,,Proc Natl Acad Sci U S A,D000595:Amino Acid Sequence; D000907:Antibodie...,...,PMC48441,1311095,1992,Cytotoxic and viral neutralizing antibodies cr...,"[The, development, of, autoimmunity, in, certa...","[Cytotoxic, and, viral, neutralizing, antibodi...","[develop, autoimmun, certain, instanc, relat, ...","[cytotox, viral, neutral, antibodi, crossreact...",[],False
5,The insulin resistance syndrome (IRS) is chara...,"Department of Epidemiology, School of Public H...",KL Edwards; MA Austin; B Newman; E Mayer; RM K...,United States,False,1049-8834,Arteriosclerosis and thrombosis : a journal of...,,Arterioscler Thromb,D001786:Blood Glucose; D001835:Body Weight; D0...,...,,7981183,1994,Multivariate analysis of the insulin resistanc...,"[The, insulin, resistance, syndrome, (, IRS, )...","[Multivariate, analysis, of, the, insulin, res...","[insulin, resist, syndrom, ir, character, cons...","[multivari, analysi, insulin, resist, syndrom,...",[],False
6,Chromatin assembly factor I (CAF-I) from human...,"Cold Spring Harbor Laboratory, New York 11724,...",PD Kaufman; R Kobayashi; N Kessler; B Stillman,United States,False,0092-8674,Cell,,Cell,D000595:Amino Acid Sequence; D001483:Base Sequ...,...,,7600578,1995,The p150 and p60 subunits of chromatin assembl...,"[Chromatin, assembly, factor, I, (, CAF-I, ), ...","[The, p150, and, p60, subunits, of, chromatin,...","[chromatin, assembl, factor, human, cell, nucl...","[subunit, chromatin, assembl, factor, molecula...",[],False
7,To examine the human antibody repertoire gener...,Eccles Program in Human Molecular Biology and ...,EE Adderson; PG Shackelford; A Quinn; PM Wilso...,United States,False,0021-9738,The Journal of clinical investigation,,J Clin Invest,D000595:Amino Acid Sequence; D000907:Antibodie...,...,PMC443339,8514881,1993,Restricted immunoglobulin VH usage and VDJ com...,"[To, examine, the, human, antibody, repertoire..."